In [1]:
import os
import numpy as np
import re
from operator import add
import dicom as dc
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from shutil import copyfile

In [2]:
def sort_by_number(list_):
    
    def atoi(text):
        return int(text) if text.isdigit() else text

    def natural_keys(text):
        return [ atoi(c) for c in re.split('(\d+)', text) ]
    ret_list=list_.sort(key = natural_keys)
    return ret_list


In [3]:
def get_number_from_list(list_):
    """
    모든 폴더에 있는 파일의 순서를 지정할수 있는 숫자를 얻어낸후 리스트로 리턴한다 
    """
    ret_list=[]
    for i,file_name in enumerate(list_):
        ret_list.append(get_number(file_name))
    ret_list=map(int ,ret_list)
    return ret_list
def get_number(file_name):
    """
    1.3.12.2.1107.5.1.4.51914.30000006121806122618700000702.dcm
    -->30000006121806122618700000702.dcm
    -->name[-3:]=702
    """
    extension=file_name.split('.')[-1]
    name=file_name.split('.')[-2]
    name=name[-3:]
    """
    """
    return name
def make_dict(list_name , list_path):
    """
    key:02 value:1.3.12.2.1107.5.1.4.51914.30000006121806122618700000702.dcm
    """
    ret_dic={}
    for i,ele in enumerate(list_path):
        ret_dic[list_name[i]]=ele
    return ret_dic
def attach_(arg1 , arg2):
    """
    arg1=elsement_wise 로 붙일 것
    arg2=list
    /home/seongjung/ASAN_project1/project1/CCM_Data/Benign/001/roimask_0013.dcm'
    """
    list_=[]
    for i in range(len(arg2)):        
        list_.append(arg1)
    list_add=map(add,list_,arg2)    
    return list_add

def find_mask(mask_dic):
    cancer_mask={}
    for i,ele in enumerate(mask_dic.keys()):

        dc_file=dc.read_file(mask_dic[ele])
        dc_img=dc_file.pixel_array
        np_img=np.asarray(dc_img)
   
        if np.mean(dc_img)>0:
            "마스크에서 종양의 여부를 식별합니다"+str(ele)+"번째에서 종양이 발견되었습니다"
            #print ele
            #print np.mean(np_img)
            cancer_mask[ele]=mask_dic[ele]
    return cancer_mask


In [4]:
def dicom_pixel(path):
    dc_file=dc.read_file(path)
    dc_img=dc_file.pixel_array
    return dc_img


In [5]:
def mask_locate(target_mask):
    locate_mask={}
    print "target_mask.keys():{}".format(target_mask.keys())
    for i,key in enumerate(target_mask.keys()):
        try:
            np_img=dicom_pixel(target_mask[key])
            row,col=np.shape(np_img)
            top_flag=False
            buttom_flag=False
            left_flag=False
            right_flag=False
            for r in range(row):
                if np.mean(np_img[r,:])>0 and top_flag==False:
                    #print r
                    top_line=np_img[r,:]>0
                    for i,ele in enumerate(top_line):
                        if ele==True:
                            top= (r,i)

                            break;

                    top_flag=True
                if top_flag==True and np.mean(np_img[r,:])==0 and buttom_flag==False:        
                    buttom_line=np_img[r-1,:]>0

                    #print r-1
                    for i,ele in enumerate(buttom_line):
                        if ele==True:
                            buttom= (r-1,i)
                            break;
                    buttom_flag=True
           

            left_flag=False
            right_flag=False
            crop_np_img=np_img[top[0]:buttom[0],:]
            for c in range(col):
                if left_flag == False and np.mean(np_img[top[0]:buttom[0],c])>0:
                    left_line = np_img[top[0]:buttom[0],c]>0
                    for i , ele in enumerate(left_line):
                        if ele==True:
                            #print i
                            left = (top[0]+i,c)
                            #print left
                            break;
                    left_flag=True
                if left_flag == True and right_flag==False and np.mean(np_img[top[0]:buttom[0],c])==0:
                    right_line =np_img[top[0]:buttom[0],c-1]>0
                    #print right_line
                    for i , ele in enumerate(right_line):
                        if ele==True:
                            #print i
                            right = (top[0]+i,c-1)
                            #print right
                            break;

                    right_flag =True
            
            center_row = ( buttom[0]-top[0])/2
            center_col = ( right[1]-left[1])/2
            center= (top[0]+center_row) , (left[1]+center_col)
            #print center
            locate_list=[top,buttom,left,right,center]
            locate_mask[key]=locate_list
        except :
            print "error info "
            print "mask : ",key
            print "mask location:",target_mask[key]
            continue;
        print 'top , buttom',top , buttom
        print 'left , right',left , right
    return locate_mask

In [6]:
a=[1,2,3]
b=[1,2,3]
attach_(1,a)

[2, 3, 4]

In [7]:
a=[1,2,3]


In [8]:
def crop_image(img , center, row,col):
    return img[center[0]-(row/2) : center[0]+(row/2) , center[1]-(col/2): center[1]+(col/2)]


In [9]:
def draw_rectangle_and_save(x,y,np_img ,save_path):

    # Create figure and axes
    fig,ax = plt.subplots(1)
    # Display the image
    ax.imshow(np_img)
    plt.savefig(save_path+'.jpeg')
    
    # Create a Rectangle patch
    rect = patches.Rectangle((x,y),50,50,fill=True,linewidth=1,edgecolor='r',facecolor='none')
    # Add the patch to the Axes
    ax.add_patch(rect)
    
    plt.savefig(save_path+'_rect.jpeg')
    plt.close(plt.gcf())


In [10]:
def crop_mask_target_images_save(mask_dic, mask_locate ,target_dic,save_path):

    print "이미지를 crop 하고 저장합니다 "
    """center 좌표를 이용해 mask 이미지와 target data의 이미지를 자른다
    mask data path 을 가지고 있는 arg: mask_dic
    center좌표를 가지고 있는 arg : mask_locate
    target data path 을 가지고 있는 arg : target_dic
    
    이미지로 저장하냐 
    numpy 로 저장 
    엔지니어 관점 문제점 파일을 2번 열어야 되서 효율성에서 떨어진다 코드를 수정해야 한다.
    넘피로도 저장을 해야 한다 .
    
    target_dic.keys()이가 항상 오름차순이라고 가정한다. 
    확인 하는 코드를 넣어야 한다.
    """
    for i ,key in enumerate(mask_dic.keys()):
        print 'key',key
        #print 'key type' ,type(key)
        key=int(key)
        mask_path=mask_dic[key]
        target_list=sorted(target_dic.keys())
        target_key=target_list[-key]
        target_key_up=target_list[key-1]
        print 'target_key 시작 위치 :'+str(target_list[0])
        print 'target_key 끝 위치 :'+str(target_list[-1])
        
        print 'target_key:'+str(target_key)
        print 'target_key_up:'+str(target_key_up)
        target_path=target_dic[target_key]
        target_path_up=target_dic[target_key_up]
        print 'target_path',target_path
        mask_img=dicom_pixel(mask_path)
        target_img=dicom_pixel(target_path)
        target_img_up=dicom_pixel(target_path_up)
        
        
        
        center=mask_locate[key][4]
        crop_mask_img=crop_image(mask_img  , center , 100 , 100)
        crop_target_img=crop_image(target_img  , center , 100 , 100)
        crop_target_img_up=crop_image(target_img_up  , center , 100 , 100)
        
        mkdir(save_path,str(target_key))
        
        draw_rec_mask_img=draw_rectangle_and_save(25,25,crop_target_img_up,save_path+'/'+str(target_key)+'/crop_target_up_'+str(target_key_up))
        draw_rec_mask_img=draw_rectangle_and_save(25,25,crop_target_img,save_path+'/'+str(target_key)+'/crop_target_'+str(target_key))
        draw_rec_target_img=draw_rectangle_and_save(25,25,crop_mask_img,save_path+'/'+str(target_key)+'/crop_mask_'+str(key))
        
        draw_rec_target_img=draw_rectangle_and_save(center[1]-25,center[0]-25,target_img_up,save_path+'/'+str(target_key)+'/target_up_'+str(target_key_up))
        draw_rec_target_img=draw_rectangle_and_save(center[1]-25,center[0]-25,target_img,save_path+'/'+str(target_key)+'/target_'+str(target_key))
        draw_rec_target_img=draw_rectangle_and_save(center[1]-25,center[0]-25,mask_img,save_path+'/'+str(target_key)+'/mask_'+str(key))
    
    
        
        
        
        
        

In [11]:
def mkdir(path,name=None , secceed_on=True):
    if os.path.isdir(path)==False:
        os.mkdir(path)
    if os.path.isdir(path)==True:
        if name != None and os.path.isdir(path+'/'+name)==False:
            os.mkdir(path+'/'+name)

In [23]:
#folder_path='/home/seongjung/ASAN_project1/project1/CCM_Data/Benign/'
#save_foler='/home/seongjung/ASAN_DATA/Benign'

folder_path='/home/user01/notebook/ASAN_DATA/Benign/'
save_foler ='/home/user01/notebook/ASAN_DATA/Benign_crop_1/'
folder_list = os.walk(folder_path).next()[1]
folder_list=sorted(folder_list) #오름 차순으로 정리 
for i,ele in enumerate(folder_list):
    try:
        print str(i)+'번째 data입니다.'
        subFolder_path=folder_path+ele
        subFolder_list=os.walk(subFolder_path).next()[1]
        print subFolder_list 
        if 'ROI_MASK' in subFolder_list:
            mask_list=os.walk(subFolder_path+'/ROI_MASK').next()[2] #...CCM_Data/Malignant/001'
            mask_path=attach_(subFolder_path+'/ROI_MASK'+'/',mask_list)
            mask_number=get_number_from_list(mask_list) #roimask_0002-->02
            mask_dic=make_dict(mask_number,mask_path)

            ct_list=os.walk(subFolder_path+'/'+subFolder_list[0]+'/').next()[2]
            ct_path=attach_(subFolder_path+'/'+subFolder_list[0]+'/',ct_list)
            ct_number=get_number_from_list(ct_list) #roimask_0002-->02
            ct_dic=make_dict(ct_number,ct_path)

            #print ct_dic.keys(); print ct_dic.values()
            cancer_mask_dic=find_mask(mask_dic)
            cancer_mask_locate=mask_locate(cancer_mask_dic)
            # cancer_mask_locate :top,buttom,left,right,center
            save_path = save_foler+'/'+ele
            if os.path.isdir(save_path)==True:
                continue;
            crop_mask_target_images_save(cancer_mask_dic, cancer_mask_locate , ct_dic ,save_path)
    except Exception as e:
        print str(e);
        continue;
        
        """
        에러가 뜨는 이유는 확실히다 처음 암 영역이 나온 이후 다시 
        나오기 때문이다. 그 알고리즈을 넣어야 한다 .
        
        """

        
        
        
        

0번째 data입니다.
['3', 'ROI_MASK']
target_mask.keys():[12]
top , buttom (320, 61) (333, 66)
left , right (324, 58) (325, 71)
이미지를 crop 하고 저장합니다 
key 12
target_key 시작 위치 :701
target_key 끝 위치 :716
target_key:705
target_key_up:712
target_path /home/user01/notebook/ASAN_DATA/Benign/001/3/1.3.12.2.1107.5.1.4.51914.30000006121806122618700000705.dcm
1번째 data입니다.
['3', 'ROI_MASK']
target_mask.keys():[10]
top , buttom (308, 209) (325, 214)
left , right (315, 206) (316, 218)
이미지를 crop 하고 저장합니다 
key 10
target_key 시작 위치 :267
target_key 끝 위치 :286
target_key:277
target_key_up:276
target_path /home/user01/notebook/ASAN_DATA/Benign/002/3/1.3.12.2.1107.5.1.4.51914.30000006022707104076500022277.dcm
2번째 data입니다.
['3', 'ROI_MASK']
target_mask.keys():[19]
error info 
mask :  19
mask location: /home/user01/notebook/ASAN_DATA/Benign/003/ROI_MASK/roimask_0019.dcm
이미지를 crop 하고 저장합니다 
key 19
target_key 시작 위치 :299
target_key 끝 위치 :334
target_key:316
target_key_up:317
target_path /home/user01/notebook/ASAN_DATA/Benig

In [26]:
#folder_path='/home/seongjung/ASAN_project1/project1/CCM_Data/Benign/'
#save_foler='/home/seongjung/ASAN_DATA/Benign'

folder_path='/home/user01/notebook/ASAN_DATA/Malignant/'
save_foler ='/home/user01/notebook/ASAN_DATA/Malignant_crop/'
folder_list = os.walk(folder_path).next()[1]
folder_list=sorted(folder_list) #오름 차순으로 정리 
for i,ele in enumerate(folder_list):
    try:
        print str(i)+'번째 data입니다.'
        subFolder_path=folder_path+ele
        subFolder_list=os.walk(subFolder_path).next()[1]
        print subFolder_list 
        if 'ROI_MASK' in subFolder_list:
            mask_list=os.walk(subFolder_path+'/ROI_MASK').next()[2] #...CCM_Data/Malignant/001'
            mask_path=attach_(subFolder_path+'/ROI_MASK'+'/',mask_list)
            mask_number=get_number_from_list(mask_list) #roimask_0002-->02
            mask_dic=make_dict(mask_number,mask_path)

            ct_list=os.walk(subFolder_path+'/'+subFolder_list[0]+'/').next()[2]
            ct_path=attach_(subFolder_path+'/'+subFolder_list[0]+'/',ct_list)
            ct_number=get_number_from_list(ct_list) #roimask_0002-->02
            ct_dic=make_dict(ct_number,ct_path)

            #print ct_dic.keys(); print ct_dic.values()
            cancer_mask_dic=find_mask(mask_dic)
            cancer_mask_locate=mask_locate(cancer_mask_dic)
            # cancer_mask_locate :top,buttom,left,right,center
            save_path = save_foler+'/'+ele
            if os.path.isdir(save_path)==True:
                continue;
            crop_mask_target_images_save(cancer_mask_dic, cancer_mask_locate , ct_dic ,save_path)
    except Exception as e:
        print str(e);
        continue;
        
        """
        에러가 뜨는 이유는 확실히다 처음 암 영역이 나온 이후 다시 
        나오기 때문이다. 그 알고리즈을 넣어야 한다 .
        
        """

        
        
        
        

0번째 data입니다.
['3', 'ROI_MASK']
target_mask.keys():[44]
top , buttom (216, 130) (233, 131)
left , right (220, 123) (220, 138)
1번째 data입니다.
['3', 'ROI_MASK']
target_mask.keys():[27, 28]
top , buttom (247, 139) (287, 123)
left , right (272, 114) (267, 153)
top , buttom (241, 137) (279, 128)
left , right (265, 112) (256, 147)
2번째 data입니다.
['8', 'ROI_MASK']
target_mask.keys():[48, 45, 46, 47]
top , buttom (268, 79) (290, 81)
left , right (275, 73) (278, 109)
top , buttom (270, 79) (288, 84)
left , right (275, 72) (277, 90)
top , buttom (266, 74) (291, 80)
left , right (273, 68) (284, 99)
top , buttom (265, 80) (293, 87)
left , right (278, 68) (278, 97)
3번째 data입니다.
['3', 'ROI_MASK']
target_mask.keys():[16, 17, 15]
top , buttom (190, 165) (215, 159)
left , right (205, 143) (201, 174)
top , buttom (186, 162) (214, 164)
left , right (200, 155) (200, 175)
top , buttom (194, 160) (213, 162)
left , right (206, 154) (204, 170)
4번째 data입니다.
['3', 'ROI_MASK']
target_mask.keys():[40, 41, 38, 39]
top 

In [131]:
def classify_files(folder_path , save_folder , include_key , notInclude_key ):
        paths=crawl_folder(folder_path)
        n_include=len(include_key)
        n_notInclude=len(notInclude_key)
        print n_include+n_notInclude
        for i,file_path in enumerate(paths): #include    
            count=0
            file_name=file_path.split('/')[-1]
            for key in include_key:
                if key in file_name:
                    count+=1
            for key_  in notInclude_key:
                if key_ not in file_name:
                    count+=1
            if count == n_include+n_notInclude:
                print file_path
                folder1=file_path.split('/')[-3]                
                folder2=file_path.split('/')[-2]
                name=file_path.split('/')[-1]
                save_name=save_folder+'/'+folder1+'_'+folder2+'_'+name
                copyfile(file_path, save_name)

In [132]:
from shutil import copyfile

In [133]:
def crawl_folder(folder_path): #search_str 찾고자 하는 string 
    search_path_list=[]
    fileList = os.walk(folder_path).next()[2]
    subFolder_list = os.walk(folder_path).next()[1]
    if(len(fileList)!=0):
        for j in range(len(fileList)):
            search_path_list.append(folder_path+'/'+fileList[j])
    if len(subFolder_list)==0:
        return search_path_list
    else: 
        for i in range(len(subFolder_list)):
            search_path_list.extend(crawl_folder(folder_path+'/'+subFolder_list[i] ))
        return search_path_list


In [134]:
os.getcwd()

'/home/user01/notebook'

In [129]:
folder_path='./ASAN_DATA/Malignant_crop'
benign_paths=crawl_folder(folder_path)
save_path= './ASAN_DATA/Malignant_from_down'
include_key=['target']
notInclude_key=['rect' ,'up','crop' ,'mask']

classify_files( folder_path , save_path , include_key , notInclude_key  )


5
1
4
2
2
4
5
./ASAN_DATA/Malignant_crop/332/516/target_516.jpeg
3
2
3
4
3
3
3
3
2
1
2
4
2
4
5
./ASAN_DATA/Malignant_crop/332/515/target_515.jpeg
3
3
4
4
1
2
3
3
4
5
./ASAN_DATA/Malignant_crop/148/952/target_952.jpeg
3
2
2
3
4
3
4
2
2
3
1
3
4
4
3
2
5
./ASAN_DATA/Malignant_crop/233/967/target_967.jpeg
5
./ASAN_DATA/Malignant_crop/233/966/target_966.jpeg
3
3
1
4
3
4
2
2
4
2
3
3
2
1
4
4
5
./ASAN_DATA/Malignant_crop/168/733/target_733.jpeg
2
4
3
3
3
2
2
3
5
./ASAN_DATA/Malignant_crop/168/732/target_732.jpeg
4
2
2
3
4
4
3
3
1
2
4
3
1
3
5
./ASAN_DATA/Malignant_crop/023/865/target_865.jpeg
3
2
3
2
4
4
4
3
4
3
4
2
1
2
3
5
./ASAN_DATA/Malignant_crop/023/866/target_866.jpeg
2
3
4
1
2
3
3
3
4
5
./ASAN_DATA/Malignant_crop/290/324/target_324.jpeg
3
4
2
2
5
./ASAN_DATA/Malignant_crop/290/323/target_323.jpeg
2
2
4
1
3
4
3
4
3
2
3
2
1
4
3
2
5
./ASAN_DATA/Malignant_crop/290/326/target_326.jpeg
4
3
2
3
4
3
3
2
3
5
./ASAN_DATA/Malignant_crop/290/325/target_325.jpeg
4
2
1
4
3
4
3
2
3
1
3
3
3
4
4
5
./ASAN_

In [141]:
folder_path='./ASAN_DATA/Benign_crop'
benign_paths=crawl_folder(folder_path)
save_path= './ASAN_DATA/benign_from_down'
include_key=['target']
notInclude_key=['rect' ,'up','crop' ,'mask']

classify_files( folder_path , save_path , include_key , notInclude_key  )


5
./ASAN_DATA/Benign_crop/023/304/target_304.jpeg
./ASAN_DATA/Benign_crop/023/303/target_303.jpeg
./ASAN_DATA/Benign_crop/023/305/target_305.jpeg
./ASAN_DATA/Benign_crop/061/18/target_18.jpeg
./ASAN_DATA/Benign_crop/061/17/target_17.jpeg
./ASAN_DATA/Benign_crop/079/517/target_517.jpeg
./ASAN_DATA/Benign_crop/038/836/target_836.jpeg
./ASAN_DATA/Benign_crop/037/407/target_407.jpeg
./ASAN_DATA/Benign_crop/064/312/target_312.jpeg
./ASAN_DATA/Benign_crop/036/443/target_443.jpeg
./ASAN_DATA/Benign_crop/002/277/target_277.jpeg
./ASAN_DATA/Benign_crop/057/598/target_598.jpeg
./ASAN_DATA/Benign_crop/041/12/target_12.jpeg
./ASAN_DATA/Benign_crop/024/421/target_421.jpeg
./ASAN_DATA/Benign_crop/024/496/target_496.jpeg
./ASAN_DATA/Benign_crop/024/427/target_427.jpeg
./ASAN_DATA/Benign_crop/024/465/target_465.jpeg
./ASAN_DATA/Benign_crop/033/783/target_783.jpeg
./ASAN_DATA/Benign_crop/066/99/target_99.jpeg
./ASAN_DATA/Benign_crop/066/100/target_100.jpeg
./ASAN_DATA/Benign_crop/066/101/target_101.jpe

In [130]:
folder_path='./ASAN_DATA/Malignant_crop'
benign_paths=crawl_folder(folder_path)
save_path= './ASAN_DATA/Malignant_from_up'
include_key=['target','up']
notInclude_key=['rect' ,'crop' ,'mask']

classify_files( folder_path , save_path , include_key , notInclude_key  )


5
0
5
./ASAN_DATA/Malignant_crop/332/516/target_up_522.jpeg
1
1
3
4
2
3
4
3
2
4
2
2
1
0
1
3
3
5
./ASAN_DATA/Malignant_crop/332/515/target_up_523.jpeg
4
4
4
3
3
0
1
2
4
3
4
4
1
3
2
5
./ASAN_DATA/Malignant_crop/148/952/target_up_918.jpeg
2
5
./ASAN_DATA/Malignant_crop/233/967/target_up_962.jpeg
3
1
4
0
4
3
3
2
1
4
4
4
2
0
3
4
5
./ASAN_DATA/Malignant_crop/233/966/target_up_963.jpeg
1
3
3
1
2
4
3
0
3
3
4
1
5
./ASAN_DATA/Malignant_crop/168/733/target_up_746.jpeg
2
4
2
1
1
4
4
3
3
1
2
5
./ASAN_DATA/Malignant_crop/168/732/target_up_747.jpeg
3
2
4
0
3
5
./ASAN_DATA/Malignant_crop/023/865/target_up_870.jpeg
2
0
4
4
4
1
2
1
3
3
5
./ASAN_DATA/Malignant_crop/023/866/target_up_869.jpeg
4
3
2
3
1
0
1
4
4
3
2
5
./ASAN_DATA/Malignant_crop/290/324/target_up_325.jpeg
0
3
4
2
2
3
4
4
3
1
1
4
3
1
5
./ASAN_DATA/Malignant_crop/290/323/target_up_326.jpeg
0
2
3
4
3
2
1
4
1
0
3
4
3
4
3
2
1
2
5
./ASAN_DATA/Malignant_crop/290/326/target_up_323.jpeg
4
2
1
4
4
5
./ASAN_DATA/Malignant_crop/290/325/target_up_324.jpe

In [149]:
folder_path='./ASAN_DATA/Benign_crop'
benign_paths=crawl_folder(folder_path)
save_path= './ASAN_DATA/benign_from_up'
include_key=['target','up']
notInclude_key=['rect' ,'crop' ,'mask']

classify_files( folder_path , save_path , include_key , notInclude_key  )


5
./ASAN_DATA/Benign_crop/023/304/target_up_279.jpeg
./ASAN_DATA/Benign_crop/023/303/target_up_280.jpeg
./ASAN_DATA/Benign_crop/023/305/target_up_278.jpeg
./ASAN_DATA/Benign_crop/061/18/target_up_981.jpeg
./ASAN_DATA/Benign_crop/061/17/target_up_982.jpeg
./ASAN_DATA/Benign_crop/079/517/target_up_472.jpeg
./ASAN_DATA/Benign_crop/038/836/target_up_833.jpeg
./ASAN_DATA/Benign_crop/037/407/target_up_408.jpeg
./ASAN_DATA/Benign_crop/064/312/target_up_302.jpeg
./ASAN_DATA/Benign_crop/036/443/target_up_436.jpeg
./ASAN_DATA/Benign_crop/002/277/target_up_276.jpeg
./ASAN_DATA/Benign_crop/057/598/target_up_604.jpeg
./ASAN_DATA/Benign_crop/041/12/target_up_9.jpeg
./ASAN_DATA/Benign_crop/024/421/target_up_496.jpeg
./ASAN_DATA/Benign_crop/024/496/target_up_421.jpeg
./ASAN_DATA/Benign_crop/024/427/target_up_465.jpeg
./ASAN_DATA/Benign_crop/024/465/target_up_427.jpeg
./ASAN_DATA/Benign_crop/033/783/target_up_777.jpeg
./ASAN_DATA/Benign_crop/066/99/target_up_125.jpeg
./ASAN_DATA/Benign_crop/066/100/tar

In [137]:
folder_path='./ASAN_DATA/Malignant_crop'
benign_paths=crawl_folder(folder_path)
save_path= './ASAN_DATA/Malignant_from_mask'
include_key=['mask']
notInclude_key=['rect' ,'crop','target']

classify_files( folder_path , save_path , include_key , notInclude_key  )


4
./ASAN_DATA/Malignant_crop/332/516/mask_36.jpeg
./ASAN_DATA/Malignant_crop/332/515/mask_37.jpeg
./ASAN_DATA/Malignant_crop/148/952/mask_18.jpeg
./ASAN_DATA/Malignant_crop/233/967/mask_33.jpeg
./ASAN_DATA/Malignant_crop/233/966/mask_34.jpeg
./ASAN_DATA/Malignant_crop/168/733/mask_42.jpeg
./ASAN_DATA/Malignant_crop/168/732/mask_43.jpeg
./ASAN_DATA/Malignant_crop/023/865/mask_38.jpeg
./ASAN_DATA/Malignant_crop/023/866/mask_37.jpeg
./ASAN_DATA/Malignant_crop/290/324/mask_32.jpeg
./ASAN_DATA/Malignant_crop/290/323/mask_33.jpeg
./ASAN_DATA/Malignant_crop/290/326/mask_30.jpeg
./ASAN_DATA/Malignant_crop/290/325/mask_31.jpeg
./ASAN_DATA/Malignant_crop/268/253/mask_32.jpeg
./ASAN_DATA/Malignant_crop/268/254/mask_31.jpeg
./ASAN_DATA/Malignant_crop/268/256/mask_29.jpeg
./ASAN_DATA/Malignant_crop/268/255/mask_30.jpeg
./ASAN_DATA/Malignant_crop/061/788/mask_13.jpeg
./ASAN_DATA/Malignant_crop/061/787/mask_14.jpeg
./ASAN_DATA/Malignant_crop/212/332/mask_37.jpeg
./ASAN_DATA/Malignant_crop/212/330/mas

In [143]:
folder_path='./ASAN_DATA/Benign_crop'
benign_paths=crawl_folder(folder_path)
save_path= './ASAN_DATA/benign_from_up'
include_key=['mask']
notInclude_key=['rect' ,'crop','target']

classify_files( folder_path , save_path , include_key , notInclude_key  )


4
./ASAN_DATA/Benign_crop/023/304/mask_25.jpeg
./ASAN_DATA/Benign_crop/023/303/mask_26.jpeg
./ASAN_DATA/Benign_crop/023/305/mask_24.jpeg
./ASAN_DATA/Benign_crop/061/18/mask_54.jpeg
./ASAN_DATA/Benign_crop/061/17/mask_55.jpeg
./ASAN_DATA/Benign_crop/079/517/mask_16.jpeg
./ASAN_DATA/Benign_crop/038/836/mask_9.jpeg
./ASAN_DATA/Benign_crop/037/407/mask_10.jpeg
./ASAN_DATA/Benign_crop/064/312/mask_21.jpeg
./ASAN_DATA/Benign_crop/036/443/mask_5.jpeg
./ASAN_DATA/Benign_crop/002/277/mask_10.jpeg
./ASAN_DATA/Benign_crop/057/598/mask_20.jpeg
./ASAN_DATA/Benign_crop/041/12/mask_8.jpeg
./ASAN_DATA/Benign_crop/024/421/mask_31.jpeg
./ASAN_DATA/Benign_crop/024/496/mask_28.jpeg
./ASAN_DATA/Benign_crop/024/427/mask_30.jpeg
./ASAN_DATA/Benign_crop/024/465/mask_29.jpeg
./ASAN_DATA/Benign_crop/033/783/mask_7.jpeg
./ASAN_DATA/Benign_crop/066/99/mask_45.jpeg
./ASAN_DATA/Benign_crop/066/100/mask_44.jpeg
./ASAN_DATA/Benign_crop/066/101/mask_43.jpeg
./ASAN_DATA/Benign_crop/025/794/mask_31.jpeg
./ASAN_DATA/Beni

In [138]:
folder_path='./ASAN_DATA/Malignant_crop'
benign_paths=crawl_folder(folder_path)
save_path= './ASAN_DATA/Malignant_crop_from_down'
include_key=['target' ,'crop' ]
notInclude_key=['rect' ,'up' ,'mask']

classify_files( folder_path , save_path , include_key , notInclude_key  )


5
./ASAN_DATA/Malignant_crop/332/516/crop_target_516.jpeg
./ASAN_DATA/Malignant_crop/332/515/crop_target_515.jpeg
./ASAN_DATA/Malignant_crop/148/952/crop_target_952.jpeg
./ASAN_DATA/Malignant_crop/233/967/crop_target_967.jpeg
./ASAN_DATA/Malignant_crop/233/966/crop_target_966.jpeg
./ASAN_DATA/Malignant_crop/168/733/crop_target_733.jpeg
./ASAN_DATA/Malignant_crop/168/732/crop_target_732.jpeg
./ASAN_DATA/Malignant_crop/023/865/crop_target_865.jpeg
./ASAN_DATA/Malignant_crop/023/866/crop_target_866.jpeg
./ASAN_DATA/Malignant_crop/290/324/crop_target_324.jpeg
./ASAN_DATA/Malignant_crop/290/323/crop_target_323.jpeg
./ASAN_DATA/Malignant_crop/290/326/crop_target_326.jpeg
./ASAN_DATA/Malignant_crop/290/325/crop_target_325.jpeg
./ASAN_DATA/Malignant_crop/268/253/crop_target_253.jpeg
./ASAN_DATA/Malignant_crop/268/254/crop_target_254.jpeg
./ASAN_DATA/Malignant_crop/268/256/crop_target_256.jpeg
./ASAN_DATA/Malignant_crop/268/255/crop_target_255.jpeg
./ASAN_DATA/Malignant_crop/061/788/crop_target

In [146]:
folder_path='./ASAN_DATA/Benign_crop'
benign_paths=crawl_folder(folder_path)
save_path= './ASAN_DATA/bengin_crop_from_down'
include_key=['target' ,'crop' ]
notInclude_key=['rect' ,'up' ,'mask']

classify_files( folder_path , save_path , include_key , notInclude_key  )


5
./ASAN_DATA/Benign_crop/023/304/crop_target_304.jpeg
./ASAN_DATA/Benign_crop/023/303/crop_target_303.jpeg
./ASAN_DATA/Benign_crop/023/305/crop_target_305.jpeg
./ASAN_DATA/Benign_crop/061/18/crop_target_18.jpeg
./ASAN_DATA/Benign_crop/061/17/crop_target_17.jpeg
./ASAN_DATA/Benign_crop/079/517/crop_target_517.jpeg
./ASAN_DATA/Benign_crop/038/836/crop_target_836.jpeg
./ASAN_DATA/Benign_crop/037/407/crop_target_407.jpeg
./ASAN_DATA/Benign_crop/064/312/crop_target_312.jpeg
./ASAN_DATA/Benign_crop/036/443/crop_target_443.jpeg
./ASAN_DATA/Benign_crop/002/277/crop_target_277.jpeg
./ASAN_DATA/Benign_crop/057/598/crop_target_598.jpeg
./ASAN_DATA/Benign_crop/041/12/crop_target_12.jpeg
./ASAN_DATA/Benign_crop/024/421/crop_target_421.jpeg
./ASAN_DATA/Benign_crop/024/496/crop_target_496.jpeg
./ASAN_DATA/Benign_crop/024/427/crop_target_427.jpeg
./ASAN_DATA/Benign_crop/024/465/crop_target_465.jpeg
./ASAN_DATA/Benign_crop/033/783/crop_target_783.jpeg
./ASAN_DATA/Benign_crop/066/99/crop_target_99.jpeg

In [139]:
folder_path='./ASAN_DATA/Malignant_crop'
benign_paths=crawl_folder(folder_path)
save_path= './ASAN_DATA/Malignant_crop_from_up'
include_key=['target' ,'crop' ,'up']
notInclude_key=['rect' ,'mask']

classify_files( folder_path , save_path , include_key , notInclude_key  )


5
./ASAN_DATA/Malignant_crop/332/516/crop_target_up_522.jpeg
./ASAN_DATA/Malignant_crop/332/515/crop_target_up_523.jpeg
./ASAN_DATA/Malignant_crop/148/952/crop_target_up_918.jpeg
./ASAN_DATA/Malignant_crop/233/967/crop_target_up_962.jpeg
./ASAN_DATA/Malignant_crop/233/966/crop_target_up_963.jpeg
./ASAN_DATA/Malignant_crop/168/733/crop_target_up_746.jpeg
./ASAN_DATA/Malignant_crop/168/732/crop_target_up_747.jpeg
./ASAN_DATA/Malignant_crop/023/865/crop_target_up_870.jpeg
./ASAN_DATA/Malignant_crop/023/866/crop_target_up_869.jpeg
./ASAN_DATA/Malignant_crop/290/324/crop_target_up_325.jpeg
./ASAN_DATA/Malignant_crop/290/323/crop_target_up_326.jpeg
./ASAN_DATA/Malignant_crop/290/326/crop_target_up_323.jpeg
./ASAN_DATA/Malignant_crop/290/325/crop_target_up_324.jpeg
./ASAN_DATA/Malignant_crop/268/253/crop_target_up_261.jpeg
./ASAN_DATA/Malignant_crop/268/254/crop_target_up_260.jpeg
./ASAN_DATA/Malignant_crop/268/256/crop_target_up_258.jpeg
./ASAN_DATA/Malignant_crop/268/255/crop_target_up_259.

In [147]:
folder_path='./ASAN_DATA/Benign_crop'
benign_paths=crawl_folder(folder_path)
save_path= './ASAN_DATA/bengin_crop_from_up'
include_key=['target' ,'crop' ,'up']
notInclude_key=['rect' ,'mask']

classify_files( folder_path , save_path , include_key , notInclude_key  )


5
./ASAN_DATA/Benign_crop/023/304/crop_target_up_279.jpeg
./ASAN_DATA/Benign_crop/023/303/crop_target_up_280.jpeg
./ASAN_DATA/Benign_crop/023/305/crop_target_up_278.jpeg
./ASAN_DATA/Benign_crop/061/18/crop_target_up_981.jpeg
./ASAN_DATA/Benign_crop/061/17/crop_target_up_982.jpeg
./ASAN_DATA/Benign_crop/079/517/crop_target_up_472.jpeg
./ASAN_DATA/Benign_crop/038/836/crop_target_up_833.jpeg
./ASAN_DATA/Benign_crop/037/407/crop_target_up_408.jpeg
./ASAN_DATA/Benign_crop/064/312/crop_target_up_302.jpeg
./ASAN_DATA/Benign_crop/036/443/crop_target_up_436.jpeg
./ASAN_DATA/Benign_crop/002/277/crop_target_up_276.jpeg
./ASAN_DATA/Benign_crop/057/598/crop_target_up_604.jpeg
./ASAN_DATA/Benign_crop/041/12/crop_target_up_9.jpeg
./ASAN_DATA/Benign_crop/024/421/crop_target_up_496.jpeg
./ASAN_DATA/Benign_crop/024/496/crop_target_up_421.jpeg
./ASAN_DATA/Benign_crop/024/427/crop_target_up_465.jpeg
./ASAN_DATA/Benign_crop/024/465/crop_target_up_427.jpeg
./ASAN_DATA/Benign_crop/033/783/crop_target_up_777.

In [140]:
folder_path='./ASAN_DATA/Malignant_crop'
benign_paths=crawl_folder(folder_path)
save_path= './ASAN_DATA/Malignant_crop_from_mask'
include_key=['mask','crop']
notInclude_key=['rect' ,'target']

classify_files( folder_path , save_path , include_key , notInclude_key  )


4
./ASAN_DATA/Malignant_crop/332/516/crop_mask_36.jpeg
./ASAN_DATA/Malignant_crop/332/515/crop_mask_37.jpeg
./ASAN_DATA/Malignant_crop/148/952/crop_mask_18.jpeg
./ASAN_DATA/Malignant_crop/233/967/crop_mask_33.jpeg
./ASAN_DATA/Malignant_crop/233/966/crop_mask_34.jpeg
./ASAN_DATA/Malignant_crop/168/733/crop_mask_42.jpeg
./ASAN_DATA/Malignant_crop/168/732/crop_mask_43.jpeg
./ASAN_DATA/Malignant_crop/023/865/crop_mask_38.jpeg
./ASAN_DATA/Malignant_crop/023/866/crop_mask_37.jpeg
./ASAN_DATA/Malignant_crop/290/324/crop_mask_32.jpeg
./ASAN_DATA/Malignant_crop/290/323/crop_mask_33.jpeg
./ASAN_DATA/Malignant_crop/290/326/crop_mask_30.jpeg
./ASAN_DATA/Malignant_crop/290/325/crop_mask_31.jpeg
./ASAN_DATA/Malignant_crop/268/253/crop_mask_32.jpeg
./ASAN_DATA/Malignant_crop/268/254/crop_mask_31.jpeg
./ASAN_DATA/Malignant_crop/268/256/crop_mask_29.jpeg
./ASAN_DATA/Malignant_crop/268/255/crop_mask_30.jpeg
./ASAN_DATA/Malignant_crop/061/788/crop_mask_13.jpeg
./ASAN_DATA/Malignant_crop/061/787/crop_mask

In [148]:
folder_path='./ASAN_DATA/Benign_crop'
benign_paths=crawl_folder(folder_path)
save_path= './ASAN_DATA/bengin_crop_from_mask'
include_key=['mask','crop']
notInclude_key=['rect' ,'target']

classify_files( folder_path , save_path , include_key , notInclude_key  )


4
./ASAN_DATA/Benign_crop/023/304/crop_mask_25.jpeg
./ASAN_DATA/Benign_crop/023/303/crop_mask_26.jpeg
./ASAN_DATA/Benign_crop/023/305/crop_mask_24.jpeg
./ASAN_DATA/Benign_crop/061/18/crop_mask_54.jpeg
./ASAN_DATA/Benign_crop/061/17/crop_mask_55.jpeg
./ASAN_DATA/Benign_crop/079/517/crop_mask_16.jpeg
./ASAN_DATA/Benign_crop/038/836/crop_mask_9.jpeg
./ASAN_DATA/Benign_crop/037/407/crop_mask_10.jpeg
./ASAN_DATA/Benign_crop/064/312/crop_mask_21.jpeg
./ASAN_DATA/Benign_crop/036/443/crop_mask_5.jpeg
./ASAN_DATA/Benign_crop/002/277/crop_mask_10.jpeg
./ASAN_DATA/Benign_crop/057/598/crop_mask_20.jpeg
./ASAN_DATA/Benign_crop/041/12/crop_mask_8.jpeg
./ASAN_DATA/Benign_crop/024/421/crop_mask_31.jpeg
./ASAN_DATA/Benign_crop/024/496/crop_mask_28.jpeg
./ASAN_DATA/Benign_crop/024/427/crop_mask_30.jpeg
./ASAN_DATA/Benign_crop/024/465/crop_mask_29.jpeg
./ASAN_DATA/Benign_crop/033/783/crop_mask_7.jpeg
./ASAN_DATA/Benign_crop/066/99/crop_mask_45.jpeg
./ASAN_DATA/Benign_crop/066/100/crop_mask_44.jpeg
./ASAN

In [21]:
def classify_files(folder_path , save_folder , include_key , notInclude_key ,notInclude_key2='up' ):
        paths=crawl_folder(folder_path)
        for i,file_path in enumerate(paths): #include
            if  include_key in file_path and notInclude_key2 not in file_path and notInclude_key not in file_path:
                
                print file_path
                folder1=file_path.split('/')[-3]                
                folder2=file_path.split('/')[-2]
                name=file_path.split('/')[-1]
                save_name=save_folder+'/'+folder1+'_'+folder2+'_'+name
                copyfile(file_path, save_name)

In [22]:
folder_path='./ASAN_DATA/Benign_crop'
benign_paths=crawl_folder(folder_path)
save_path= './ASAN_DATA/Benign_crop_mask'
include_key='crop_mask'
notInclude_key='rect'

classify_files( folder_path , save_path , include_key , notInclude_key  )


./ASAN_DATA/Benign_crop/023/51/crop_mask_24.jpeg
./ASAN_DATA/Benign_crop/023/49/crop_mask_26.jpeg
./ASAN_DATA/Benign_crop/023/50/crop_mask_25.jpeg
./ASAN_DATA/Benign_crop/061/18/crop_mask_55.jpeg
./ASAN_DATA/Benign_crop/061/19/crop_mask_54.jpeg
./ASAN_DATA/Benign_crop/079/61/crop_mask_16.jpeg
./ASAN_DATA/Benign_crop/038/12/crop_mask_9.jpeg
./ASAN_DATA/Benign_crop/037/9/crop_mask_10.jpeg
./ASAN_DATA/Benign_crop/064/31/crop_mask_21.jpeg
./ASAN_DATA/Benign_crop/036/12/crop_mask_5.jpeg
./ASAN_DATA/Benign_crop/002/11/crop_mask_10.jpeg
./ASAN_DATA/Benign_crop/057/14/crop_mask_20.jpeg
./ASAN_DATA/Benign_crop/041/11/crop_mask_8.jpeg
./ASAN_DATA/Benign_crop/024/29/crop_mask_30.jpeg
./ASAN_DATA/Benign_crop/024/28/crop_mask_31.jpeg
./ASAN_DATA/Benign_crop/024/31/crop_mask_28.jpeg
./ASAN_DATA/Benign_crop/024/30/crop_mask_29.jpeg
./ASAN_DATA/Benign_crop/033/13/crop_mask_7.jpeg
./ASAN_DATA/Benign_crop/066/20/crop_mask_44.jpeg
./ASAN_DATA/Benign_crop/066/19/crop_mask_45.jpeg
./ASAN_DATA/Benign_crop/0